# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## Installing dependencies

In [1]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-e3shdxjv
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-e3shdxjv
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit d125b79bb7bfdeab751f93bac37039950fe24ce5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done


In [2]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-01-23 10:00:12--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14711 (14K) [text/plain]
Saving to: ‘structured_qa.csv’

structured_qa.csv   100%[===================>]  14.37K  --.-KB/s    in 0.003s  

2025-01-23 10:00:19 (5.28 MB/s) - ‘structured_qa.csv’ saved [14711/14711]



# Setup

In [10]:
import os
import google.generativeai as genai

GEMINI_API_KEY = None
if not GEMINI_API_KEY:
    raise ValueError("Please set the GEMINI_API_KEY variable to your API key")
os.environ["LOGURU_LEVEL"] = "INFO"
genai.configure(api_key=GEMINI_API_KEY)

## Function to Download Document

In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

from loguru import logger


def download_document(url, output_file):
    if not Path(output_file).exists():
        urlretrieve(url, output_file)
        logger.info(f"Downloaded {url} to {output_file}")
    else:
        logger.info(f"File {output_file} already exists")

## Function to Process a single Document

In [14]:
import json
import time


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Uploading file")
    file = genai.upload_file(document_file, mime_type="application/pdf")
    while file.state.name == "PROCESSING":
        logger.debug("Waiting for file to be processed.")
        time.sleep(2)
        file = genai.get_file(file.name)

    logger.info("Predicting")
    n = 0
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        if n > 0 and n % 9 == 0:
            logger.info("Waiting for 60 seconds")
            time.sleep(60)
        question = row["question"]
        logger.info(f"Question: {question}")
        response = model.model.generate_content([file, question])
        logger.info(response.text)
        response_json = json.loads(response.text)
        answers[index] = response_json["answer"]
        sections[index] = response_json["section"]
        n += 1
    return answers, sections

## Load Model

In [ ]:
from structured_qa.model_loaders import load_gemini_model

In [7]:
FULL_CONTEXT_PROMPT = """
You are given an input document and a question.
You can only answer the question based on the information in the document.
You will return a JSON name with two keys: "section" and "answer".
In `"section"`, you will return the name of the section where you found the answer.
In `"answer"`, you will return the answer one of the following JSON:
- Yes/No (for boolean questions)
Is the model an LLM?
{
  "section": "1. Introduction",
  "answer": "No"
}
- Single number (for numeric questions)
How many layers does the model have?
{
  "section": "2. Architecture",
  "answer": 12
}
- Single letter (for multiple-choice questions)
What is the activation function used in the model?
-A: ReLU
-B: Sigmoid
-C: Tanh
{
  "section": "2. Architecture",
  "answer": "C"
}
"""

In [8]:
model = load_gemini_model(
    "gemini-2.0-flash-exp",
    system_prompt=FULL_CONTEXT_PROMPT,
    generation_config={
        "response_mime_type": "application/json",
    },
)

# Run Benchmark

In [15]:
import pandas as pd


logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)

for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    download_document(document_link, downloaded_document)

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-01-23 10:03:52.899 | INFO     | __main__:<cell line: 0>:4 - Loading input data
2025-01-23 10:03:52.906 | INFO     | __main__:<cell line: 0>:10 - Downloading document https://arxiv.org/pdf/1706.03762
2025-01-23 10:03:52.908 | INFO     | __main__:process_document:12 - Uploading file
2025-01-23 10:03:56.426 | INFO     | __main__:process_document:19 - Predicting
2025-01-23 10:03:56.428 | INFO     | __main__:process_document:28 - Question: What type of architecture does the model use? -A: decoder only -B: encoder only -C: encoder-decoder
2025-01-23 10:04:03.326 | INFO     | __main__:process_document:30 - {
  "section": "3 Model Architecture",
  "answer": "C"
}
2025-01-23 10:04:03.352 | INFO     | __main__:process_document:28 - Question: How many layers compose the encoder?
2025-01-23 10:04:10.125 | INFO     | __main__:process_document:30 - {
  "section": "3.1 Encoder and Decoder Stacks",
  "answer": 6
}
2025-01-23 10:04:10.129 | INFO     | __main__:process_document:28 - Question: How m

# Results

In [16]:
results = pd.read_csv("results.csv")
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,section,question,answer,pred_answer,pred_section
5,5,https://arxiv.org/pdf/1706.03762,3 Model Architecture,Does the final model use learned positional em...,NO,YES,6.2 Model Variations
13,13,https://arxiv.org/pdf/2210.05189,3 Experimental Results,How many parameters are in the y = x^2 toy mod...,14,39,Table 1. Computation and memory analysis of to...
18,18,https://arxiv.org/pdf/2106.09685v2.pdf,5.5 Scaling Up to GPT-3,How much memory is saved (in GB) when training...,850,0.85,4. Practical Benefits and Limitations.
22,22,https://eur-lex.europa.eu/legal-content/EN/TXT...,Prohibited AI Practices (Article 5),Which type of AI systems are banned by the AI ...,C,B,Article 5
39,39,https://authorsalliance.org/wp-content/uploads...,Chapter 5 Where do you want to make your work ...,Are Gold Open Access and Green Open Access mut...,NO,YES,Chapter 5
74,74,https://commission.europa.eu/document/download...,Natural lighting,What is the daylight factor required for façad...,0.7,0.7%,4. VISUAL COMFORT


In [17]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.9210526315789473